In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [5]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [8]:
data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [12]:
data1 = data[data['Diagnosis'] == 'normal']
data2 = data[data['Diagnosis'] == 'early neoplasia']
data3 = data[data['Diagnosis'] == 'cancer']

In [27]:
data1.columns[2:]

Index(['LOC643837', 'LOC100130417', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1',
       'C1orf170', 'HES4', 'ISG15', 'AGRN',
       ...
       'CLIC2', 'RPS4Y1', 'ZFY', 'PRKY', 'USP9Y', 'DDX3Y', 'CD24', 'CYorf15B',
       'KDM5D', 'EIF1AY'],
      dtype='object', length=15748)

In [22]:
import scipy.stats as sts

print('Проверяем нормальные и раннюю')
count12 = 0
Pvalues1 = []
for i in data.columns[2:]:
    Pvalue = sts.ttest_ind(data1[i], data2[i], equal_var = False).pvalue
    Pvalues1.append(Pvalue)
    if Pvalue < 0.05:
        count12 +=1
        
print('Проверяем раннюю и рак')
Pvalues2 = []
count23 = 0
for i in data.columns[2:]:
    Pvalue = sts.ttest_ind(data2[i], data3[i], equal_var = False).pvalue
    Pvalues2.append(Pvalue)
    if Pvalue < 0.05:
        count23 +=1

Проверяем нормальные и раннюю
Проверяем раннюю и рак


In [19]:
def write_answer(task,ans):
    with open("bio_" + str(task)+".txt", "w") as fout:
        fout.write(str(ans))

In [20]:
write_answer(1.1, count12)
write_answer(1.2, count23)

# task 2

In [50]:
import statsmodels.stats.multitest as smm

In [51]:
reject_1, p_corrected_1, a1_1, a2_1 = smm.multipletests(Pvalues1, 
                                            alpha = 0.05/2, 
                                            method = 'holm') 

reject_2, p_corrected_2, a1_2, a2_2 = smm.multipletests(Pvalues2, 
                                            alpha = 0.05/2, 
                                            method = 'holm')

In [52]:
import numpy as np

def fold_change(control, treatment):
    C, T = np.mean(control), np.mean(treatment)
    if T>C:
        return T/C
    else:
        return (-1)*C/T

count1 = 0
List = data1.columns[2:]
for i in range(len(Pvalues1)):
    if p_corrected_1[i] < 0.025:
        a = fold_change(np.array(data1[List[i]]), np.array(data2[List[i]])) 
        if abs(a) > 1.5:
            count1 += 1

In [53]:
write_answer(2.1, count1)

In [54]:
count2 = 0
List = data2.columns[2:]
for i in range(len(Pvalues1)):
    if p_corrected_2[i] < 0.025:
        a = fold_change(np.array(data2[List[i]]), np.array(data3[List[i]])) 
        if abs(a) > 1.5:
            count2 += 1

In [55]:
write_answer(2.2, count2)

# task 3

In [56]:
reject_1b, p_corrected_1b, a1_1b, a2_1b = smm.multipletests(Pvalues1, 
                                            alpha = 0.05/2, 
                                            method = 'fdr_bh') 

reject_2b, p_corrected_2b, a1_2b, a2_2b = smm.multipletests(Pvalues2, 
                                            alpha = 0.05/2, 
                                            method = 'fdr_bh')

In [57]:
count1b = 0

for i in range(len(Pvalues1)):
    if p_corrected_1b[i] < 0.025:
        a = fold_change(np.array(data1[List[i]]), np.array(data2[List[i]])) 
        if abs(a) > 1.5:
            count1b += 1
            
count2b = 0
for i in range(len(Pvalues1)):
    if p_corrected_2b[i] < 0.025:
        a = fold_change(np.array(data2[List[i]]), np.array(data3[List[i]])) 
        if abs(a) > 1.5:
            count2b += 1

In [58]:
write_answer(3.1, count1b)

In [59]:
write_answer(3.2, count2b)